In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier

from plotly.offline import init_notebook_mode, iplot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

import warnings

We have addded libraries which we will use later.

In [ ]:
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

train = pd.read_csv("/kaggle/input/datathon2023/train.csv")
test = pd.read_csv("/kaggle/input/datathon2023/test_x.csv")
train.drop(columns="index",inplace=True)
test.drop(columns="index",inplace=True)

We have included Train and Test datasets to workspace with the help of pandas library.

In [ ]:
train.sample(5)

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.isna().sum()

In [ ]:
print('Çift girişli gözlemler: {}'.format(train.duplicated().sum()))

We checked the values whether missing or not. Also for gather information about dataset, we have used info(), describe() and isna() functions. Based on the results of the functions which we had used, we observed the valus are whether missing or not. 

In [ ]:
#Order Rate
train["Siparişe Dönüşüm Oranı"] = train["Yıllık Ortalama Sipariş Verilen Ürün Adedi"] / train["Yıllık Ortalama Sepete Atılan Ürün Adedi"]
train["Siparişe Dönüşüm Oranı"] = train["Siparişe Dönüşüm Oranı"].apply(lambda x: 0 if x == np.inf else x)

test["Siparişe Dönüşüm Oranı"] = test["Yıllık Ortalama Sipariş Verilen Ürün Adedi"] / test["Yıllık Ortalama Sepete Atılan Ürün Adedi"]
test["Siparişe Dönüşüm Oranı"] = test["Siparişe Dönüşüm Oranı"].apply(lambda x: 0 if x == np.inf else x)
   
#Customer Spent Rate
train["Harcama Gücü"] = train["Yıllık Ortalama Gelir"] / train["Yıllık Ortalama Satın Alım Miktarı"]
test["Harcama Gücü"] = test["Yıllık Ortalama Gelir"] / test["Yıllık Ortalama Satın Alım Miktarı"]
train.head()  

"Siparişe Dönüşüm Oranı"(Means Order Rate) and "Harcama Gücü"(Means Spending Power) and we calculate the values that are important in the e-commerce sector by using the data we have.

Now we will group our dataset according to the values in the "Öbek Adı" data column and start to examine which features are weighted for each cluster group. We will apply these procedures for each phrase class respectively. We used Bar Graphs for categorical data and Line Graphs for continuous data. We also performed correlation calculation to check whether there is a relationship between continuous data and visualised it with the heatmap graph of seaborn library. 

In [ ]:
train_grouped_obek1= train[train['Öbek İsmi'] == 'obek_1']
train_grouped_obek1

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 1 Cinsiyet Dağılımı')
train_grouped_obek1['Cinsiyet'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 1 Yaş Grubu Dağılımı')
train_grouped_obek1['Yaş Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 1 Medeni Durum Dağılımı')
train_grouped_obek1['Medeni Durum'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 1 Eğitim Düzeyi Dağılımı')
train_grouped_obek1['Eğitim Düzeyi'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 1 İstihdam Durumu Dağılımı')
train_grouped_obek1['İstihdam Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 1 Yaşadığı Şehir Dağılımı')
train_grouped_obek1['Yaşadığı Şehir'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 1 Ürün Grubu Dağılımı')
train_grouped_obek1['En Çok İlgilendiği Ürün Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 1 Eğitime Devam Durumu Dağılımı')
train_grouped_obek1['Eğitime Devam Etme Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

In [ ]:
train_grouped_obek1['Yıllık Ortalama Gelir'].plot(cmap = 'cividis')
plt.title('Öbek 1 Yıllık Ortalama Gelir Dağılımı')
plt.show()

train_grouped_obek1['Yıllık Ortalama Satın Alım Miktarı'].plot(cmap = 'cividis')
plt.title('Öbek 1 Yıllık Ortalama Satın Alım Miktarı Grafiği')
plt.show()

train_grouped_obek1['Yıllık Ortalama Sipariş Verilen Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 1 Yıllık Ortalama Sipariş Verilen Ürün Adedi Grafiği')
plt.show()

train_grouped_obek1['Yıllık Ortalama Sepete Atılan Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 1 Yıllık Ortalama Sepete Atılan Ürün Adedi Grafiği')
plt.show()

train_grouped_obek1['Siparişe Dönüşüm Oranı'].plot(cmap = 'cividis')
plt.title('Öbek 1 Sepete Atılan Ürünün Siparişe Dönüşüm Oranı Grafiği')
plt.show()

train_grouped_obek1['Harcama Gücü'].plot(cmap = 'cividis')
plt.title('Öbek 1 Harcama Gücü Grafiği')
plt.show()


data_correlation_calc = train_grouped_obek1[['Yıllık Ortalama Gelir', 'Yıllık Ortalama Satın Alım Miktarı', 'Yıllık Ortalama Sipariş Verilen Ürün Adedi', 'Yıllık Ortalama Sepete Atılan Ürün Adedi', 'Siparişe Dönüşüm Oranı', 'Harcama Gücü']]
corr_matrix = data_correlation_calc.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='mako')
plt.title('Öbek 1 Korelasyon Matrisi')
plt.show()

In [ ]:
train_grouped_obek2= train[train['Öbek İsmi'] == 'obek_2']
train_grouped_obek2

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 2 Cinsiyet Dağılımı')
train_grouped_obek2['Cinsiyet'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 2 Yaş Grubu Dağılımı')
train_grouped_obek2['Yaş Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 2 Medeni Durum Dağılımı')
train_grouped_obek2['Medeni Durum'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 2 Eğitim Düzeyi Dağılımı')
train_grouped_obek2['Eğitim Düzeyi'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 2 İstihdam Durumu Dağılımı')
train_grouped_obek2['İstihdam Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 2 Yaşadığı Şehir Dağılımı')
train_grouped_obek2['Yaşadığı Şehir'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 2 Ürün Grubu Dağılımı')
train_grouped_obek2['En Çok İlgilendiği Ürün Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 2 Eğitime Devam Durumu Dağılımı')
train_grouped_obek2['Eğitime Devam Etme Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

In [ ]:
train_grouped_obek2['Yıllık Ortalama Gelir'].plot(cmap = 'cividis')
plt.title('Öbek 2 Yıllık Ortalama Gelir Dağılımı')
plt.show()

train_grouped_obek2['Yıllık Ortalama Satın Alım Miktarı'].plot(cmap = 'cividis')
plt.title('Öbek 2 Yıllık Ortalama Satın Alım Miktarı Grafiği')
plt.show()

train_grouped_obek2['Yıllık Ortalama Sipariş Verilen Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 2 Yıllık Ortalama Sipariş Verilen Ürün Adedi Grafiği')
plt.show()

train_grouped_obek2['Yıllık Ortalama Sepete Atılan Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 2 Yıllık Ortalama Sepete Atılan Ürün Adedi Grafiği')
plt.show()

train_grouped_obek2['Siparişe Dönüşüm Oranı'].plot(cmap = 'cividis')
plt.title('Öbek 2 Sepete Atılan Ürünün Siparişe Dönüşüm Oranı Grafiği')
plt.show()

train_grouped_obek2['Harcama Gücü'].plot(cmap = 'cividis')
plt.title('Öbek 2 Harcama Gücü Grafiği')
plt.show()


data_correlation_calc = train_grouped_obek2[['Yıllık Ortalama Gelir', 'Yıllık Ortalama Satın Alım Miktarı', 'Yıllık Ortalama Sipariş Verilen Ürün Adedi', 'Yıllık Ortalama Sepete Atılan Ürün Adedi', 'Siparişe Dönüşüm Oranı', 'Harcama Gücü']]
corr_matrix = data_correlation_calc.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='mako')
plt.title('Öbek 2 Korelasyon Matrisi')
plt.show()

In [ ]:
train_grouped_obek3= train[train['Öbek İsmi'] == 'obek_3']
train_grouped_obek3

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 3 Cinsiyet Dağılımı')
train_grouped_obek3['Cinsiyet'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 3 Yaş Grubu Dağılımı')
train_grouped_obek3['Yaş Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 3 Medeni Durum Dağılımı')
train_grouped_obek3['Medeni Durum'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 3 Eğitim Düzeyi Dağılımı')
train_grouped_obek3['Eğitim Düzeyi'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 3 İstihdam Durumu Dağılımı')
train_grouped_obek3['İstihdam Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 3 Yaşadığı Şehir Dağılımı')
train_grouped_obek3['Yaşadığı Şehir'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 3 Ürün Grubu Dağılımı')
train_grouped_obek3['En Çok İlgilendiği Ürün Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 3 Eğitime Devam Durumu Dağılımı')
train_grouped_obek3['Eğitime Devam Etme Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

In [ ]:
train_grouped_obek3['Yıllık Ortalama Gelir'].plot(cmap = 'cividis')
plt.title('Öbek 3 Yıllık Ortalama Gelir Dağılımı')
plt.show()

train_grouped_obek3['Yıllık Ortalama Satın Alım Miktarı'].plot(cmap = 'cividis')
plt.title('Öbek 3 Yıllık Ortalama Satın Alım Miktarı Grafiği')
plt.show()

train_grouped_obek3['Yıllık Ortalama Sipariş Verilen Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 3 Yıllık Ortalama Sipariş Verilen Ürün Adedi Grafiği')
plt.show()

train_grouped_obek3['Yıllık Ortalama Sepete Atılan Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 3 Yıllık Ortalama Sepete Atılan Ürün Adedi Grafiği')
plt.show()

train_grouped_obek3['Siparişe Dönüşüm Oranı'].plot(cmap = 'cividis')
plt.title('Öbek 3 Sepete Atılan Ürünün Siparişe Dönüşüm Oranı Grafiği')
plt.show()

train_grouped_obek3['Harcama Gücü'].plot(cmap = 'cividis')
plt.title('Öbek 3 Harcama Gücü Grafiği')
plt.show()


data_correlation_calc = train_grouped_obek3[['Yıllık Ortalama Gelir', 'Yıllık Ortalama Satın Alım Miktarı', 'Yıllık Ortalama Sipariş Verilen Ürün Adedi', 'Yıllık Ortalama Sepete Atılan Ürün Adedi', 'Siparişe Dönüşüm Oranı', 'Harcama Gücü']]
corr_matrix = data_correlation_calc.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='mako')
plt.title('Öbek 3 Korelasyon Matrisi')
plt.show()

In [ ]:
train_grouped_obek4= train[train['Öbek İsmi'] == 'obek_4']
train_grouped_obek4

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 4 Cinsiyet Dağılımı')
train_grouped_obek4['Cinsiyet'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 4 Yaş Grubu Dağılımı')
train_grouped_obek4['Yaş Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 4 Medeni Durum Dağılımı')
train_grouped_obek4['Medeni Durum'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 4 Eğitim Düzeyi Dağılımı')
train_grouped_obek4['Eğitim Düzeyi'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 4 İstihdam Durumu Dağılımı')
train_grouped_obek4['İstihdam Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 4 Yaşadığı Şehir Dağılımı')
train_grouped_obek4['Yaşadığı Şehir'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 4 Ürün Grubu Dağılımı')
train_grouped_obek4['En Çok İlgilendiği Ürün Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 4 Eğitime Devam Durumu Dağılımı')
train_grouped_obek4['Eğitime Devam Etme Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

In [ ]:
train_grouped_obek4['Yıllık Ortalama Gelir'].plot(cmap = 'cividis')
plt.title('Öbek 4 Yıllık Ortalama Gelir Dağılımı')
plt.show()

train_grouped_obek4['Yıllık Ortalama Satın Alım Miktarı'].plot(cmap = 'cividis')
plt.title('Öbek 4 Yıllık Ortalama Satın Alım Miktarı Grafiği')
plt.show()

train_grouped_obek4['Yıllık Ortalama Sipariş Verilen Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 4 Yıllık Ortalama Sipariş Verilen Ürün Adedi Grafiği')
plt.show()

train_grouped_obek4['Yıllık Ortalama Sepete Atılan Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 4 Yıllık Ortalama Sepete Atılan Ürün Adedi Grafiği')
plt.show()

train_grouped_obek4['Siparişe Dönüşüm Oranı'].plot(cmap = 'cividis')
plt.title('Öbek 4 Sepete Atılan Ürünün Siparişe Dönüşüm Oranı Grafiği')
plt.show()

train_grouped_obek4['Harcama Gücü'].plot(cmap = 'cividis')
plt.title('Öbek 4 Harcama Gücü Grafiği')
plt.show()


data_correlation_calc = train_grouped_obek4[['Yıllık Ortalama Gelir', 'Yıllık Ortalama Satın Alım Miktarı', 'Yıllık Ortalama Sipariş Verilen Ürün Adedi', 'Yıllık Ortalama Sepete Atılan Ürün Adedi', 'Siparişe Dönüşüm Oranı', 'Harcama Gücü']]
corr_matrix = data_correlation_calc.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='mako')
plt.title('Öbek 4 Korelasyon Matrisi')
plt.show()

In [ ]:
train_grouped_obek5= train[train['Öbek İsmi'] == 'obek_5']
train_grouped_obek5

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 5 Cinsiyet Dağılımı')
train_grouped_obek5['Cinsiyet'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 5 Yaş Grubu Dağılımı')
train_grouped_obek5['Yaş Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 5 Medeni Durum Dağılımı')
train_grouped_obek5['Medeni Durum'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 5 Eğitim Düzeyi Dağılımı')
train_grouped_obek5['Eğitim Düzeyi'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 5 İstihdam Durumu Dağılımı')
train_grouped_obek5['İstihdam Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 5 Yaşadığı Şehir Dağılımı')
train_grouped_obek5['Yaşadığı Şehir'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 5 Ürün Grubu Dağılımı')
train_grouped_obek5['En Çok İlgilendiği Ürün Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 5 Eğitime Devam Durumu Dağılımı')
train_grouped_obek5['Eğitime Devam Etme Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

In [ ]:
train_grouped_obek5['Yıllık Ortalama Gelir'].plot(cmap = 'cividis')
plt.title('Öbek 5 Yıllık Ortalama Gelir Dağılımı')
plt.show()

train_grouped_obek5['Yıllık Ortalama Satın Alım Miktarı'].plot(cmap = 'cividis')
plt.title('Öbek 5 Yıllık Ortalama Satın Alım Miktarı Grafiği')
plt.show()

train_grouped_obek5['Yıllık Ortalama Sipariş Verilen Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 5 Yıllık Ortalama Sipariş Verilen Ürün Adedi Grafiği')
plt.show()

train_grouped_obek5['Yıllık Ortalama Sepete Atılan Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 5 Yıllık Ortalama Sepete Atılan Ürün Adedi Grafiği')
plt.show()

train_grouped_obek5['Siparişe Dönüşüm Oranı'].plot(cmap = 'cividis')
plt.title('Öbek 5 Sepete Atılan Ürünün Siparişe Dönüşüm Oranı Grafiği')
plt.show()

train_grouped_obek5['Harcama Gücü'].plot(cmap = 'cividis')
plt.title('Öbek 5 Harcama Gücü Grafiği')
plt.show()


data_correlation_calc = train_grouped_obek5[['Yıllık Ortalama Gelir', 'Yıllık Ortalama Satın Alım Miktarı', 'Yıllık Ortalama Sipariş Verilen Ürün Adedi', 'Yıllık Ortalama Sepete Atılan Ürün Adedi', 'Siparişe Dönüşüm Oranı', 'Harcama Gücü']]
corr_matrix = data_correlation_calc.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='mako')
plt.title('Öbek 5 Korelasyon Matrisi')
plt.show()

In [ ]:
train_grouped_obek6= train[train['Öbek İsmi'] == 'obek_6']
train_grouped_obek6

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 6 Cinsiyet Dağılımı')
train_grouped_obek6['Cinsiyet'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 6 Yaş Grubu Dağılımı')
train_grouped_obek6['Yaş Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 6 Medeni Durum Dağılımı')
train_grouped_obek6['Medeni Durum'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 6 Eğitim Düzeyi Dağılımı')
train_grouped_obek6['Eğitim Düzeyi'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 6 İstihdam Durumu Dağılımı')
train_grouped_obek6['İstihdam Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 6 Yaşadığı Şehir Dağılımı')
train_grouped_obek6['Yaşadığı Şehir'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 6 Ürün Grubu Dağılımı')
train_grouped_obek6['En Çok İlgilendiği Ürün Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 6 Eğitime Devam Durumu Dağılımı')
train_grouped_obek6['Eğitime Devam Etme Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

In [ ]:
train_grouped_obek6['Yıllık Ortalama Gelir'].plot(cmap = 'cividis')
plt.title('Öbek 6 Yıllık Ortalama Gelir Dağılımı')
plt.show()

train_grouped_obek6['Yıllık Ortalama Satın Alım Miktarı'].plot(cmap = 'cividis')
plt.title('Öbek 6 Yıllık Ortalama Satın Alım Miktarı Grafiği')
plt.show()

train_grouped_obek6['Yıllık Ortalama Sipariş Verilen Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 6 Yıllık Ortalama Sipariş Verilen Ürün Adedi Grafiği')
plt.show()

train_grouped_obek6['Yıllık Ortalama Sepete Atılan Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 6 Yıllık Ortalama Sepete Atılan Ürün Adedi Grafiği')
plt.show()

train_grouped_obek6['Siparişe Dönüşüm Oranı'].plot(cmap = 'cividis')
plt.title('Öbek 6 Sepete Atılan Ürünün Siparişe Dönüşüm Oranı Grafiği')
plt.show()

train_grouped_obek6['Harcama Gücü'].plot(cmap = 'cividis')
plt.title('Öbek 6 Harcama Gücü Grafiği ')
plt.show()


data_correlation_calc = train_grouped_obek6[['Yıllık Ortalama Gelir', 'Yıllık Ortalama Satın Alım Miktarı', 'Yıllık Ortalama Sipariş Verilen Ürün Adedi', 'Yıllık Ortalama Sepete Atılan Ürün Adedi', 'Siparişe Dönüşüm Oranı', 'Harcama Gücü']]
corr_matrix = data_correlation_calc.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='mako')
plt.title('Öbek 6 Korelasyon Matrisi')
plt.show()

In [ ]:
train_grouped_obek7= train[train['Öbek İsmi'] == 'obek_7']
train_grouped_obek7

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 7 Cinsiyet Dağılımı')
train_grouped_obek7['Cinsiyet'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 7 Yaş Grubu Dağılımı')
train_grouped_obek7['Yaş Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 7 Medeni Durum Dağılımı')
train_grouped_obek7['Medeni Durum'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 7 Eğitim Düzeyi Dağılımı')
train_grouped_obek7['Eğitim Düzeyi'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 7 İstihdam Durumu Dağılımı')
train_grouped_obek7['İstihdam Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 7 Yaşadığı Şehir Dağılımı')
train_grouped_obek7['Yaşadığı Şehir'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 7 Ürün Grubu Dağılımı')
train_grouped_obek7['En Çok İlgilendiği Ürün Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 7 Eğitime Devam Durumu Dağılımı')
train_grouped_obek7['Eğitime Devam Etme Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

In [ ]:
train_grouped_obek7['Yıllık Ortalama Gelir'].plot(cmap = 'cividis')
plt.title('Öbek 7 Yıllık Ortalama Gelir Dağılımı')
plt.show()

train_grouped_obek7['Yıllık Ortalama Satın Alım Miktarı'].plot(cmap = 'cividis')
plt.title('Öbek 7 Yıllık Ortalama Satın Alım Miktarı Grafiği')
plt.show()

train_grouped_obek7['Yıllık Ortalama Sipariş Verilen Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 7 Yıllık Ortalama Sipariş Verilen Ürün Adedi Grafiği')
plt.show()

train_grouped_obek7['Yıllık Ortalama Sepete Atılan Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 7 Yıllık Ortalama Sepete Atılan Ürün Adedi Grafiği')
plt.show()

train_grouped_obek7['Siparişe Dönüşüm Oranı'].plot(cmap = 'cividis')
plt.title('Öbek 7 Sepete Atılan Ürünün Siparişe Dönüşüm Oranı Grafiği')
plt.show()

train_grouped_obek7['Harcama Gücü'].plot(cmap = 'cividis')
plt.title('Öbek 7 Harcama Gücü Grafiği')
plt.show()


data_correlation_calc = train_grouped_obek7[['Yıllık Ortalama Gelir', 'Yıllık Ortalama Satın Alım Miktarı', 'Yıllık Ortalama Sipariş Verilen Ürün Adedi', 'Yıllık Ortalama Sepete Atılan Ürün Adedi', 'Siparişe Dönüşüm Oranı', 'Harcama Gücü']]
corr_matrix = data_correlation_calc.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='mako')
plt.title('Öbek 7 Korelasyon Matrisi')
plt.show()

In [ ]:
train_grouped_obek8= train[train['Öbek İsmi'] == 'obek_8']
train_grouped_obek8

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 8 Cinsiyet Dağılımı')
train_grouped_obek8['Cinsiyet'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 8 Yaş Grubu Dağılımı')
train_grouped_obek8['Yaş Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 8 Medeni Durum Dağılımı')
train_grouped_obek8['Medeni Durum'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 8 Eğitim Düzeyi Dağılımı')
train_grouped_obek8['Eğitim Düzeyi'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 8 İstihdam Durumu Dağılımı')
train_grouped_obek8['İstihdam Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 8 Yaşadığı Şehir Dağılımı')
train_grouped_obek8['Yaşadığı Şehir'].value_counts().plot(kind = 'bar', cmap = 'cividis')

fig, ax = plt.subplots(1, 2)
fig.tight_layout()
plt.subplot(1, 2, 1)
plt.title('Öbek 8 Ürün Grubu Dağılımı')
train_grouped_obek8['En Çok İlgilendiği Ürün Grubu'].value_counts().plot(kind = 'bar', cmap = 'cividis')
plt.subplot(1, 2, 2)
plt.title('Öbek 8 Eğitime Devam Durumu Dağılımı')
train_grouped_obek8['Eğitime Devam Etme Durumu'].value_counts().plot(kind = 'bar', cmap = 'cividis')

In [ ]:
train_grouped_obek8['Yıllık Ortalama Gelir'].plot(cmap = 'cividis')
plt.title('Öbek 8 Yıllık Ortalama Gelir Dağılımı')
plt.show()

train_grouped_obek8['Yıllık Ortalama Satın Alım Miktarı'].plot(cmap = 'cividis')
plt.title('Öbek 8 Yıllık Ortalama Satın Alım Miktarı Grafiği')
plt.show()

train_grouped_obek8['Yıllık Ortalama Sipariş Verilen Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 8 Yıllık Ortalama Sipariş Verilen Ürün Adedi Grafiği')
plt.show()

train_grouped_obek8['Yıllık Ortalama Sepete Atılan Ürün Adedi'].plot(cmap = 'cividis')
plt.title('Öbek 8 Yıllık Ortalama Sepete Atılan Ürün Adedi Grafiği')
plt.show()

train_grouped_obek8['Siparişe Dönüşüm Oranı'].plot(cmap = 'cividis')
plt.title('Öbek 8 Sepete Atılan Ürünün Siparişe Dönüşüm Oranı Grafiği')
plt.show()

train_grouped_obek8['Harcama Gücü'].plot(cmap = 'cividis')
plt.title('Öbek 8 Harcama Gücü Grafiği')
plt.show()


data_correlation_calc = train_grouped_obek8[['Yıllık Ortalama Gelir', 'Yıllık Ortalama Satın Alım Miktarı', 'Yıllık Ortalama Sipariş Verilen Ürün Adedi', 'Yıllık Ortalama Sepete Atılan Ürün Adedi', 'Siparişe Dönüşüm Oranı', 'Harcama Gücü']]
corr_matrix = data_correlation_calc.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='mako')
plt.title('Öbek 8 Korelasyon Matrisi')
plt.show()

In [ ]:
new_cat = ['Yıllık Ortalama Gelir','Yıllık Ortalama Satın Alım Miktarı','Yıllık Ortalama Sepete Atılan Ürün Adedi']
new_cat_name = ['Gelir Grubu','Sadakat(Sipariş)','Aktivite(Sepet)']
for i, newcat in enumerate(new_cat):
    quantity_mean = train[newcat].mean()
    quantity_std = train[newcat].std()

    low_limit_quantity = quantity_mean - quantity_std
    high_limit_quantity = quantity_mean + quantity_std

    def categorized_quantity(quantity):
        if quantity <= low_limit_quantity:
            return 'Düşük'
        elif quantity <= high_limit_quantity:
            return 'Orta'
        else:
            return 'Yüksek'

    train[new_cat_name[i]] = train[newcat].apply(categorized_quantity) 
    test[new_cat_name[i]] = test[newcat].apply(categorized_quantity) 


We have categorised the continuous data of "Yıllık Ortalama Gelir"(Means Annual Average Income), "Yıllık Ortalama Satın Alım Miktarı"(Means Annual Average Purchase Amount) and "Yıllık Ortalama Sepete Atılan Ürün Adedi"(Means Annual Average Number of Products Added to the Basket) into three different categories using the mean and standard deviation values. These categories are also used in the e-commerce sector and give us information about the habits of customers:
* Income Group
* Loyalty(Order)
* Activity(Basket).

We use three different data values for these categories: "Low", "Medium" and "High". We will use this categorical data in classification algorithms.

In [ ]:
num_cols = ['Yıllık Ortalama Gelir','Yıllık Ortalama Satın Alım Miktarı','Yıllık Ortalama Sipariş Verilen Ürün Adedi','Yıllık Ortalama Sepete Atılan Ürün Adedi']
cat_cols = [col for col in train.columns if train[col].dtypes == "O"]
cat_cols.remove("Öbek İsmi")
tag = "Öbek İsmi"

We separate the categorical and numeric columns of our dataset. We also remove the "Öbek İsmi" column from the categorical columns. 

Now we use pie chart and bar charts to analyse how the data in our dataset is distributed among the clusters.

In [ ]:
color_palette = px.colors.qualitative.T10

trace1 = go.Pie(values = train['Öbek İsmi'].value_counts(),labels = train['Öbek İsmi'].unique(),textinfo='percent',insidetextorientation='radial',legendgroup='show')
trace2 = go.Histogram(x = train['Öbek İsmi'],marker=dict(color=color_palette))

fig = make_subplots(rows=1, cols=2,subplot_titles=('Pie chart','Bar chart'),specs=[[{"type": "pie"}, {"type": "histogram"}]])  

                          
fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)
fig['layout'].update(title='Müşteri Dağılımları',)
fig.show();

For each categorical data, we perform the feature analysis on the basis of phrases.

In [ ]:
for col in train[cat_cols].columns:
    
    fig = px.histogram(train,
                   x=col,
                   color="Öbek İsmi",
                   barmode="group",
                   title=("{} ve Öbek İsmi").format(col),
                   color_discrete_sequence=px.colors.qualitative.T10)  # İstediğiniz renk şemasını seçebilirsiniz
    fig.show()

Here, we visualised the outliers in our dataset by calculating their 1st and 3rd quartile values. We used a box plot to show the position of the outliers against the ordered data.

In [ ]:
for col in num_cols:
    Q1 = train[col].quantile(0.25)
    Q3 = train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_fence = Q1 - 1.5 * IQR
    upper_fence = Q3 + 1.5 * IQR
    
    if col =="Yıllık Ortalama Gelir":
        outliers = train[col][(train[col] < lower_fence) | (train[col] > upper_fence)]
        outliers_indices = train[col][(train[col] < lower_fence) | (train[col] > upper_fence)].dropna(how='all').index
    
    upper_fence_count = len(train[train[col] > upper_fence])

    data = [go.Box(x=train[col]),]
    layout = go.Layout(title= col)
    layout = go.Layout(title=col + f"<br>({upper_fence_count} adet anomali gözlem)")

    fig = go.Figure(data = data, layout = layout)
    fig.show()

We sorted our continuous data and show their frequency in our data set. We also showed the statistical values for each data feature with the Describe() function.

In [ ]:
for col in num_cols:
    print(col)
    print(train[col].describe().T)
    print()

    plt.figure(figsize=(8, 6))
    sns.histplot(train[col], kde=True)
    plt.title('Histogram of '+col)
    plt.xlabel(col)
    plt.ylabel('Frekans')
    plt.show()

We checked whether there is a significant relationship between them by calculating correlation for numerical data features and visualised it with the heatmap.

In [ ]:
sns.heatmap(train[num_cols].corr(),annot=True,cmap='mako',vmin=-1,vmax=1)

We checked whether there is a significant relationship between them by calculating correlation for numerical data features and visualised it with the heatmap.

In [ ]:
train['Eğitim Düzeyi'].unique()

In [ ]:
label_encoder = LabelEncoder()
train['Eğitim Düzeyi Encoded'] = label_encoder.fit_transform(train['Eğitim Düzeyi'])

education_mapping = {
    'Eğitimsiz': 0,
    'İlkokul Mezunu': 1,
    'Ortaokul Mezunu': 2,
    'Lise Mezunu': 3,
    'Yüksekokul Mezunu': 4,
    'Üniversite Mezunu': 5,
    'Yüksek Lisans Mezunu': 6,
    'Doktora Mezunu': 7,
    'Doktora Ötesi': 8
}

train.drop(columns="Eğitim Düzeyi Encoded", inplace=True)
train['Eğitim Düzeyi'] = train['Eğitim Düzeyi'].map(education_mapping)

new_cat_columns = train[new_cat_name]
encoder = OrdinalEncoder()

encoded_data = encoder.fit_transform(new_cat_columns)

encoded_df = pd.DataFrame(encoded_data, columns=new_cat_name)
train.drop(columns={'Gelir Grubu','Sadakat(Sipariş)','Aktivite(Sepet)'},inplace=True)
train = pd.concat([train, encoded_df], axis=1)
train[['Gelir Grubu', 'Sadakat(Sipariş)', 'Aktivite(Sepet)']] = train[['Gelir Grubu', 'Sadakat(Sipariş)', 'Aktivite(Sepet)']].astype(int)

We repeat the same process for the test dataset.

In [ ]:
label_encoder = LabelEncoder()
test['Eğitim Düzeyi Encoded'] = label_encoder.fit_transform(test['Eğitim Düzeyi'])

education_mapping = {
    'Eğitimsiz': 0,
    'İlkokul Mezunu': 1,
    'Ortaokul Mezunu': 2,
    'Lise Mezunu': 3,
    'Yüksekokul Mezunu': 4,
    'Üniversite Mezunu': 5,
    'Yüksek Lisans Mezunu': 6,
    'Doktora Mezunu': 7,
    'Doktora Ötesi': 8
}

test.drop(columns="Eğitim Düzeyi Encoded", inplace=True)
test['Eğitim Düzeyi'] = test['Eğitim Düzeyi'].map(education_mapping)

new_cat_columns = test[new_cat_name]
encoder = OrdinalEncoder()

encoded_data = encoder.fit_transform(new_cat_columns)

encoded_df = pd.DataFrame(encoded_data, columns=new_cat_name)
test.drop(columns={'Gelir Grubu','Sadakat(Sipariş)','Aktivite(Sepet)'},inplace=True)
test = pd.concat([test, encoded_df], axis=1)


Here, using the get_dummies() method of the Pandas library, we create a new column for each special data in the categorical columns and assign its data 0 or 1. The reason for doing this is that the dataset needs to be clean and standardised for Machine Learning algorithms to give highly accurate results. With this method, we have turned our categorical data into columns and digitised their values as binary.

In [ ]:
train = pd.get_dummies(train, columns=['Cinsiyet','Yaş Grubu','Medeni Durum','Yaşadığı Şehir', 'İstihdam Durumu','Eğitime Devam Etme Durumu', 'En Çok İlgilendiği Ürün Grubu'],drop_first=True)
train = train.replace({True: 1, False: 0})

test = pd.get_dummies(test, columns=['Cinsiyet','Yaş Grubu','Medeni Durum','Yaşadığı Şehir', 'İstihdam Durumu','Eğitime Devam Etme Durumu', 'En Çok İlgilendiği Ürün Grubu'],drop_first=True)
test = test.replace({True: 1, False: 0})

We use the MinMaxScaler() method to standardise continuous data. With this method, we standardise the values between 0 and 1 to facilitate the learning process of Machine Learning algorithms.

In [ ]:
scaler = MinMaxScaler()

float64_cols = train.select_dtypes(include=['float64'])
train[float64_cols.columns] = scaler.fit_transform(float64_cols)

float64_cols = test.select_dtypes(include=['float64'])
test[float64_cols.columns] = scaler.fit_transform(float64_cols)

train

In [ ]:
X = train.drop(columns=['Öbek İsmi','Siparişe Dönüşüm Oranı','Harcama Gücü'])
test = test.drop(columns=['Siparişe Dönüşüm Oranı','Harcama Gücü'])
y = train['Öbek İsmi']

In [ ]:
X

We apply the training data to our model using the Random Trees Classification Algorithm, we aimed to obtain an average validation score by observing how the model works on different data within the same dataset by Cross Validation. Cross Validation divides the training dataset into the desired number of parts and trains these data chunks separately to the same model. We then calculated the accuracy of the model for each chunk of data and averaged these values to see how accurate the model was in our dataset. Finally, we saved the prediction results of our model for later comparison.

In [ ]:
best_rf_model = RandomForestClassifier( max_depth=10, min_samples_leaf=1, 
                                       min_samples_split=5, n_estimators=200, 
                                       random_state=42)

# Cross-validation ile modelin performansını değerlendirelim
# 5 katlı çapraz doğrulama işlemi:
cv_scores = cross_val_score(best_rf_model, X, y, cv=5)

# Her bir katın doğruluk değerlerini yazdıralım.
for fold, score in enumerate(cv_scores, start=1):
    print(f"Fold {fold}: Doğruluk = {score:.4f}")

# Ortalama doğruluk değerini hesaplayalım
mean_accuracy = cv_scores.mean()
print()
print("Ortalama Doğruluk:", mean_accuracy)

best_rf_model.fit(X, y)
y_pred_rf = best_rf_model.predict(X)
accuracy = accuracy_score(y, y_pred_rf)

In [ ]:
#3 standart sapmanın üzerindeki değerleri anomali kabul ederek anomalilerin hatalı tahminlerini görüntüleyelim.
for i in outliers_indices:
    mismatched_rows = train[train.index == i][y_pred_rf != train['Öbek İsmi']]
mismatched_rows

Here we have shown which column affects the model result by using the feature_importances_ value of the Random Trees Algorithm. In this example, the column "Yıllık Ortalama Sipariş Verilen Ürün Adedi" is the most effective data feature for the Random Trees Algorithm.

In [ ]:
# Özellik önemi değerlerini inceleyelim.
feature_importances = best_rf_model.feature_importances_

# Özellik isimleri
feature_names = X.columns

# Bar grafiğini oluştur
fig = go.Figure(data=[go.Bar(x=feature_names, y=feature_importances)])
fig.update_layout(title='Özellik Önemi Değerlendirmesi (Random Forest)',
                  xaxis=dict(title='Özellikler'),
                  yaxis=dict(title='Önem Değerleri'),
                 height=700)
fig.update_xaxes(tickangle=45) 
# Grafiği göster
fig.show()

This time we repeat the same steps on the XGBoost Algorithm and save the results of our model for later comparison.

In [ ]:
best_xgb_model = XGBClassifier(colsample_bytree=0.22, gamma=0.05, learning_rate=0.2,
                                max_depth=3, min_child_weight=1, n_estimators=28,
                                reg_alpha=1, reg_lambda=1, subsample=0.8,
                                objective='binary:logistic', booster='gbtree')
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# Cross-validation ile modelin performansını değerlendireceğiz.
# 5 katlı çapraz doğrulama yapıyoruz.
cv_scores = cross_val_score(best_xgb_model, X, y_encoded, cv=5)

# Her bir katın doğruluk değerlerini yazdıralım.
for fold, score in enumerate(cv_scores, start=1):
    print(f"Fold {fold}: Doğruluk = {score:.4f}")

# Ortalama doğruluk değerini hesaplayalım.
mean_accuracy = cv_scores.mean()
print()
print("Ortalama Doğruluk:", mean_accuracy)


best_xgb_model.fit(X, y_encoded)
y_pred_xgb = best_xgb_model.predict(X)

In this example, the data feature "Sadakat(Alışveriş)" column was the most effective data feature for the XGBoost Algorithm. 

In [ ]:
# Özellik önemi değerlerini inceleyelim.
feature_importances = best_xgb_model.feature_importances_

# Özellik isimleri
feature_names = X.columns

# Sütun grafiğini oluşturalım.
fig = go.Figure(data=[go.Bar(x=feature_names, y=feature_importances)])
fig.update_layout(title='Özellik Önemi Değerlendirmesi (XGBClassifier)',
                  xaxis=dict(title='Özellikler'),
                  yaxis=dict(title='Önem Değerleri'),
                 height=700)
fig.update_xaxes(tickangle=45) 
# Grafiği göster
fig.show()

We repeat the same steps for another classification algorithm, the LightGBM Algorithm, and save the results for comparison.

In [ ]:
best_lgbm_model = LGBMClassifier(
    learning_rate=0.1,
    max_depth=3,
    n_estimators=100,
    subsample=0.8,
    colsample_bytree=0.8
)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Cross-validation ile LGBM modelinin performansını değerlendirelim.
cv_scores_lgbm = cross_val_score(best_lgbm_model, X, y_encoded, cv=5)

# Her bir katın doğruluk değerlerini yazdıralım.
for fold, score in enumerate(cv_scores_lgbm, start=1):
    print(f"Fold {fold}: Doğruluk = {score:.4f}")

# Ortalama doğruluk değerini hesaplayalım.
mean_accuracy_lgbm = cv_scores_lgbm.mean()
print()
print("Ortalama Doğruluk (LGBM):", mean_accuracy_lgbm)

# Modeli tam veri üzerinde eğiterek tahmin yapalım.
best_lgbm_model.fit(X, y)
y_pred = best_lgbm_model.predict(X)
accuracy = accuracy_score(y, y_pred)

y_pred_lgbm = y_pred

We have shown in the column graph that the "Yıllık Ortalama Sipariş Verilen Ürün Adedi" feature is the most effective data feature for the LightGBM Algorithm.

In [ ]:
# Özellik önemi değerlerini inceleyelim.
feature_importances = best_lgbm_model.feature_importances_

# Özellik isimleri
feature_names = X.columns

# Sütun grafiğini oluşturalım.
fig = go.Figure(data=[go.Bar(x=feature_names, y=feature_importances)])
fig.update_layout(title='Özellik Önemi Değerlendirmesi (XGBClassifier)',
                  xaxis=dict(title='Özellikler'),
                  yaxis=dict(title='Önem Değerleri'),
                 height=700)
fig.update_xaxes(tickangle=45) 
# Grafiği göster
fig.show()

We restore the "Öbek İsmi" column that we have digitised in order to be able to interpret the results.

In [ ]:
obek_mapping = {
    0: 'obek_1',
    1: 'obek_2',
    2: 'obek_3',
    3: 'obek_4',
    4: 'obek_5',
    5: 'obek_6',
    6: 'obek_7',
    7: 'obek_8'
}

y_pred_xgb = [obek_mapping[prediction] for prediction in y_pred_xgb]

Let's collect the predictions produced by our 3 classification algorithms in a single data set and analyse the predictions of the algorithms.

In [ ]:
data = {
    'id': range(X.shape[0]),
    'Öbek İsmi rf': y_pred_rf
}
Prediction_Dataframe = pd.DataFrame(data)
Prediction_Dataframe.set_index('id', inplace=True)
Prediction_Dataframe['Öbek İsmi xgb'] = y_pred_xgb
Prediction_Dataframe['Öbek İsmi lgbm'] = y_pred_lgbm
Prediction_Dataframe['Öbek İsmi gerçek'] = train['Öbek İsmi']
Prediction_Dataframe

Here we will apply the Ensemble Method, that is, we choose the most common prediction value among the results found by the 3 algorithms as the result of our models. We do this in order to determine the most accurate result among the prediction results of our models.

In [ ]:
#ensemble - Tahminler arası karşılaştırmayla 3 algoritmanın tahminleri arasında en sık rastlanan değeri buluyoruz.
Prediction_Dataframe['Öbek İsmi ensemble'] = Prediction_Dataframe.mode(axis=1).iloc[:, 0]

We filtered the different prediction results given by our algorithms.

In [ ]:
#3 farklı algoritma ile bulunan tahminlerimizin kontrolü
different_rows = Prediction_Dataframe[ 
    (Prediction_Dataframe['Öbek İsmi xgb'] != Prediction_Dataframe['Öbek İsmi rf']) | (Prediction_Dataframe['Öbek İsmi xgb'] != Prediction_Dataframe['Öbek İsmi lgbm']) | (Prediction_Dataframe['Öbek İsmi lgbm'] != Prediction_Dataframe['Öbek İsmi rf'])
]
different_rows

In [ ]:
#Tahminlerimizin gerçek değerler ile kıyaslanmasına bakalım.
different_rows = Prediction_Dataframe[ 
    (Prediction_Dataframe['Öbek İsmi ensemble'] != train['Öbek İsmi'])
]
different_rows

In [ ]:
 #Eğer üç sütun farklı değerlere sahipse 'Öbek İsmi xgb' değerini 'Öbek İsmi ensemble' sütununa ekliyoruz.
conditions = ((Prediction_Dataframe['Öbek İsmi rf'] != Prediction_Dataframe['Öbek İsmi lgbm']) &
              (Prediction_Dataframe['Öbek İsmi lgbm'] != Prediction_Dataframe['Öbek İsmi xgb']) &
              (Prediction_Dataframe['Öbek İsmi rf'] != Prediction_Dataframe['Öbek İsmi xgb']))

# Her satır için koşulların sağlanıp sağlanmadığını kontrol ederek işlem yap
Prediction_Dataframe['Öbek İsmi ensemble'] = np.where(conditions, Prediction_Dataframe['Öbek İsmi xgb'], Prediction_Dataframe['Öbek İsmi ensemble'])

After applying the Ensemble Method, we compare the results of our models with the highest accuracy with the real data in the training set. If all three algorithms predicted different values, we used the result of the XGBoost Algorithm as the result of the Ensemble Method. As a result of our tests and experiments, it was seen that the XGBoost algorithm produced more accurate predictions on data where all three algorithms produced different predictions.

In [ ]:
#Tahminlerimizin gerçek değerler ile kıayslamasını yaptık.
different_rows = Prediction_Dataframe[ 
    (Prediction_Dataframe['Öbek İsmi ensemble'] != train['Öbek İsmi'])
]
different_rows

We start making predictions to the algorithms using the test data.

In [ ]:
#TEST dataframe ile tahmin yapıyoruz

#RandomForestClassifier
best_rf_model = RandomForestClassifier( max_depth=10, min_samples_leaf=1, 
                                       min_samples_split=5, n_estimators=200, 
                                       random_state=42)

best_rf_model.fit(X, y)
y_pred_rf_test = best_rf_model.predict(test)

#XGBClassifier
best_xgb_model = XGBClassifier(colsample_bytree=0.22, gamma=0.05, learning_rate=0.2,
                                max_depth=3, min_child_weight=1, n_estimators=28,
                                reg_alpha=1, reg_lambda=1, subsample=0.8,
                                objective='binary:logistic', booster='gbtree')
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

best_xgb_model.fit(X, y_encoded)
y_pred_xgb_test = best_xgb_model.predict(test)

y_pred_xgb_test = [obek_mapping[prediction] for prediction in y_pred_xgb_test]

#LightGBM Classifier
best_lgbm_model = LGBMClassifier(
    learning_rate=0.1,
    max_depth=3,        
    n_estimators=100,  
    subsample=0.8,      
    colsample_bytree=0.8  
)

# Cross-validation ile modelin performansını değerlendiriyoruz.
# Örneğin, 5 katlı çapraz doğrulama:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


y_encoded_reshaped = y_encoded.reshape(-1, 1)
best_lgbm_model.fit(X, y_encoded)

y_pred_lgbm = best_lgbm_model.predict(X)
y_pred_lgbm_test = best_lgbm_model.predict(test)
y_pred_lgbm_test  = [obek_mapping[prediction] for prediction in y_pred_lgbm_test]
y_pred_lgbm = [obek_mapping[prediction] for prediction in y_pred_lgbm]
print(accuracy_score(y, y_pred_lgbm))

We will combine the prediction results of the three algorithms in a single dataset and apply the Ensemble Method. Again, if the three models find different results in the Ensemble Method, we will use the result of the XGBoost Algorithm as the result of the Ensemble Method.

In [ ]:
test_data = {
    'id': range(y_pred_rf_test.shape[0]),
    'Öbek İsmi rf': y_pred_rf_test
}
test_dataframe = pd.DataFrame(test_data)
test_dataframe['Öbek İsmi lgbm'] = y_pred_lgbm_test
test_dataframe['Öbek İsmi xgb'] = y_pred_xgb_test
test_dataframe.set_index('id', inplace=True)
test_dataframe

In [ ]:
#ensemble - Tahminler arası karşılaştırmayla 3 algoritma arasından en sık rastlanan tahmin değerini bulunuyoruz.
test_dataframe['Öbek İsmi ensemble'] = test_dataframe.mode(axis=1).iloc[:, 0]

In [ ]:
 #Eğer üç sütun farklı değerlere sahipse 'Öbek İsmi xgb' değerini 'Öbek İsmi ensemble' sütununa yazdırıyoruz.
conditions = ((test_dataframe['Öbek İsmi rf'] != test_dataframe['Öbek İsmi lgbm']) &
              (test_dataframe['Öbek İsmi lgbm'] != test_dataframe['Öbek İsmi xgb']) &
              (test_dataframe['Öbek İsmi rf'] != test_dataframe['Öbek İsmi xgb']))

# Her satır için koşulların sağlanıp sağlanmadığını kontrol ediyoruz.
test_dataframe['Öbek İsmi ensemble'] = np.where(conditions, test_dataframe['Öbek İsmi xgb'], test_dataframe['Öbek İsmi ensemble'])

We prepared the obtained prediction results for submission by placing them in the dataset.

In [ ]:
test_dataframe['Öbek İsmi ensemble'].to_csv('submission-son.csv')